# ConvGRU

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable

In [2]:
class ConvRNN(nn.Module):
    def __init__(self,
                 cell_param,
                 return_state,
                 return_sequence,
                 ):
        super(ConvRNN, self).__init__()
        self.cell_param = cell_param
        self.return_state =return_state
        self.return_sequence = return_sequence

    def init_paramter(self,shape):
        return Variable(torch.zeros(shape).cuda())

    def forward(self, input, state=None):

        if state is None:
            state = self.init_hidden(input)
        else:
            state = state
        # batch_size
        self.minibatch_size = input.size()[0]
        # time_step
        self.n_step = input.size()[1]
        # outputs = Variable(torch.zeros(self.minibatch_size, self.n_step, self.cell_param['output_channels'], input.size()[-2], input.size()[-1]).cuda())
        outputs = []
        # 按时间步处理
        for i in range(self.n_step):
            x_t = input[:, i, :, :, :]
            state = self.cell(x_t, state)
            if type(state) == type((1,2)): 
                outputs.append(state[0]) 
            else:
                outputs.append(state)
        outputs = torch.stack(outputs,dim=1)
        self.outputs = outputs
        if self.return_sequence:
            if self.return_state:
                return outputs, state
            else:
                return outputs
        else:
            if self.return_state:
                return state
            else:
                if type(state) == type((1)): # int
                    return state[0]
                else:
                    return state


class ConvRNNCell(nn.Module):
    def __init__(self, cell_param):
        super(ConvRNNCell, self).__init__()
        self.input_dim = cell_param['input_channels']
        self.output_dim = cell_param['output_channels']
        self.input_to_state_kernel_size = cell_param['input_to_state_kernel_size']
        self.state_to_state_kernel_size = cell_param['state_to_state_kernel_size']

In [3]:
class MultiConvGRU(ConvRNN):
    def __init__(self,
                 cell_param,
                 return_state,
                 return_sequence):
        super(MultiConvGRU, self).__init__(
            cell_param,
            return_state,
            return_sequence
        )

        self.build()

    def build(self):
        self.cell = MConvGRUCell(self.cell_param)

    def init_hidden(self, input):

        hidden_size = (input.size()[0], self.cell_param['output_channels'], input.size()[-2], input.size()[-1])
        h = Variable(self.init_paramter(hidden_size))
        state = h

        return state



class MConvGRUCell(ConvRNNCell):
    def __init__(self, cell_param):
        super(MConvGRUCell, self).__init__(cell_param)
        self.build_model()

    def get_parameter(self,shape,init_method = 'xavier'):
        param = Parameter(torch.Tensor(*shape).cuda())
        if init_method == 'xavier':
            nn.init.xavier_uniform_(param)
        elif init_method == 'zero':
            nn.init.constant_(param,0)
        else:
            raise ('init method error')
        return param

    def build_model(self):

        input_to_state_shape = [
            self.output_dim,
            self.input_dim,
            self.input_to_state_kernel_size[0],
            self.input_to_state_kernel_size[1]
        ]
        state_to_state_shape = [
            self.output_dim,
            self.output_dim,
            self.state_to_state_kernel_size[0],
            self.state_to_state_kernel_size[1]
        ]
        state_bias_shape = [
            1, self.output_dim, 1, 1
        ]

        self.w_xz1 = self.get_parameter(input_to_state_shape)
        self.w_xz2 = self.get_parameter(state_to_state_shape)
        self.w_xz3 = self.get_parameter(state_to_state_shape)
        self.w_hz = self.get_parameter(state_to_state_shape)

        self.w_xr1 = self.get_parameter(input_to_state_shape)
        self.w_xr2 = self.get_parameter(state_to_state_shape)
        self.w_xr3 = self.get_parameter(state_to_state_shape)
        self.w_hr = self.get_parameter(state_to_state_shape)

        self.w_xh1 = self.get_parameter(input_to_state_shape)
        self.w_xh2 = self.get_parameter(state_to_state_shape)
        self.w_xh3 = self.get_parameter(state_to_state_shape)
        self.w_hh = self.get_parameter(state_to_state_shape)



        self.b_z = self.get_parameter(state_bias_shape,'zero')
        self.b_r = self.get_parameter(state_bias_shape,'zero')
        self.b_h_ = self.get_parameter(state_bias_shape,'zero')

    def same_padding(self,kernel_size):
        if kernel_size[0]%2==0 or kernel_size[1]%2==0:
            raise('The kernel size must to be odd if you want padding!')
        else:
            padding = tuple((int((kernel_size[0]-1)/2),int((kernel_size[1]-1)/2)))
        return padding


    def cell(self, x_t, hidden):
        h_tm1 = hidden
        Z = torch.sigmoid(
            F.conv2d(h_tm1, self.w_hz, bias=None, padding=self.same_padding(self.state_to_state_kernel_size))
            + F.conv2d(
                F.conv2d(
                    F.conv2d(
                        x_t,
                        self.w_xz1,
                        bias=None,
                        padding=self.same_padding(self.input_to_state_kernel_size)
                    ),
                    self.w_xz2,
                    bias = None,
                    padding = self.same_padding(self.state_to_state_kernel_size)
                ),
                self.w_xz3,
                bias = None,
                padding = self.same_padding(self.state_to_state_kernel_size)
            )
            + self.b_z
        )

        R = torch.sigmoid(
            F.conv2d(h_tm1, self.w_hr, bias=None, padding=self.same_padding(self.state_to_state_kernel_size))
            + F.conv2d(
                F.conv2d(
                    F.conv2d(
                        x_t,
                        self.w_xr1,
                        bias=None,
                        padding=self.same_padding(self.input_to_state_kernel_size)
                    ),
                    self.w_xr2,
                    bias = None,
                    padding = self.same_padding(self.state_to_state_kernel_size)
                ),
                self.w_xr3,
                bias = None,
                padding = self.same_padding(self.state_to_state_kernel_size)
            )
            + self.b_r
        )

        H_ = F.leaky_relu(
            F.conv2d(h_tm1,self.w_hh,bias = None,padding = self.same_padding(self.state_to_state_kernel_size))
            + R*F.conv2d(
                    F.conv2d(
                        F.conv2d(
                            x_t,
                            self.w_xh1,
                            bias=None,
                            padding = self.same_padding(self.input_to_state_kernel_size)
                        ),
                        self.w_xh2,
                        bias = None,
                        padding = self.same_padding(self.state_to_state_kernel_size)
                    ),
                    self.w_xh3,
                    bias = None,
                    padding = self.same_padding(self.state_to_state_kernel_size)
                )
            + self.b_h_,negative_slope = 0.2
        )

        H = (1-Z)*H_ + Z*h_tm1

        return H

    def forward(self, input, hidden):
        h_t = self.cell(input,hidden)
        return h_t

# Channel Attention 

In [4]:
## Channel Attention (CA) Layer
class CALayer(nn.Sequential):
    def __init__(self, channel, reduction=16):
        super(CALayer, self).__init__()
        self.conv_du = nn.Sequential(
                # global average pooling: feature --> point
                nn.AdaptiveAvgPool2d(1),
                # feature channel downscale and upscale --> channel weight
                nn.Conv2d(channel, channel // reduction, 1, padding=0, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // reduction, channel, 1, padding=0, bias=True),
                nn.Sigmoid()
        )

    def forward(self, x):
        y = self.conv_du(x)
        return x * y


# Read Reanalysis Data

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from geopy.distance import great_circle

import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.autograd import Variable
# from torchsummary import summary
import datetime

import os
import random

# forecast 24-hour lead time 
pre_seq = 4
batch_size = 128
epochs = 128
min_val_loss = 100
model_name = './model_saver/Model.pkl'
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [6]:
train = pd.read_csv('./data/CMA_train_'+str(pre_seq*6)+'h.csv', header=None)
test= pd.read_csv('./data/CMA_test_'+str(pre_seq*6)+'h.csv', header=None)

In [7]:
train.shape, test.shape

((8406, 59), (2747, 59))

In [8]:
CLIPER_feature =  pd.concat((train, test), axis=0)
CLIPER_feature.reset_index(drop=True, inplace=True)

In [11]:
X_wide_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_wide = X_wide_scaler.fit_transform(CLIPER_feature.iloc[:, 6:])
X_wide_train = X_wide[0: train.shape[0], :]

y = y_scaler.fit_transform(CLIPER_feature.loc[:, 3:4])
y_train = y[0: train.shape[0], :]

In [12]:
reanalysis_type = 'z'
# 0 means now 
# 1 means 6-hour ago
# 2 means 12-hour ago
ahead_times = [0,1,2,3]
pressures = [1000, 750, 500, 250]
sequential_reanalysis_list = []
reanalysis_test_dict = {}
X_deep_scaler_dict = {}

In [13]:
for ahead_time in ahead_times:

    reanalysis_list = []
    for pressure in pressures:
        
        folder = None
        if ahead_time == 0:
            folder = reanalysis_type
        else:
            folder = reanalysis_type + '_' + str(ahead_time*6)
        train_reanalysis_csv = pd.read_csv('./data/ERA_Interim/'+folder+'/'+reanalysis_type+str(pressure)+'_train_31_31.csv', header=None)
        test_reanalysis_csv = pd.read_csv('./data/ERA_Interim/'+folder+'/'+reanalysis_type+str(pressure)+'_test_31_31.csv', header=None)
        
        train_reanalysis = train_reanalysis_csv[train_reanalysis_csv[0].isin(train[0].unique())]
        test_reanalysis = test_reanalysis_csv[test_reanalysis_csv[0].isin(test[0].unique())]
        reanalysis_test_dict[reanalysis_type+str(pressure)+str(ahead_time)] = test_reanalysis
        
        reanalysis =  pd.concat((train_reanalysis, test_reanalysis), axis=0)
        reanalysis.reset_index(drop=True, inplace=True)
        
        scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
        X_deep_scaler_dict[scaler_name] = MinMaxScaler()
        X_deep = X_deep_scaler_dict[scaler_name] .fit_transform(reanalysis.loc[:, 5:])
        
        X_deep_final = X_deep[0: train.shape[0], :].reshape(-1, 1, 1, 31, 31)
        reanalysis_list.append(X_deep_final)
    
    X_deep_temp = np.concatenate(reanalysis_list[:], axis=2)
    print("ahead_time:", ahead_time, X_deep_temp.shape)
    sequential_reanalysis_list.append(X_deep_temp)

X_deep_train = np.concatenate(sequential_reanalysis_list, axis=1)

ahead_time: 0 (8406, 1, 4, 31, 31)
ahead_time: 1 (8406, 1, 4, 31, 31)
ahead_time: 2 (8406, 1, 4, 31, 31)
ahead_time: 3 (8406, 1, 4, 31, 31)


In [14]:
X_wide_train.shape, X_deep_train.shape

((8406, 53), (8406, 4, 4, 31, 31))

# Training set and Validation set

In [15]:
class TrainLoader(Data.Dataset):
    def __init__(self, X_wide_train, X_deep_train, y_train):
        self.X_wide_train = X_wide_train
        self.X_deep_train = X_deep_train
        self.y_train = y_train
        
    def __getitem__(self, index):
        return [self.X_wide_train[index], self.X_deep_train[index]], self.y_train[index]
    
    def __len__(self):
        return len(self.X_wide_train)

In [16]:
full_train_index = [*range(0, len(X_wide_train))]

train_index, val_index, _, _, = train_test_split(full_train_index,full_train_index,test_size=0.1)

In [17]:
len(train_index), len(val_index)

(7565, 841)

In [18]:
train_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[train_index], X_deep_train[train_index], y_train[train_index]), 
                                                 batch_size=batch_size, shuffle=True)

In [19]:
val_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[val_index], X_deep_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size, shuffle=True)

# Wide & Deep Model

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.channel_attention = CALayer(4, reduction=2)
        
        self.convGRU1 = MultiConvGRU(cell_param={
            'input_channels' : 4,
            'output_channels' : 64,
            'input_to_state_kernel_size' : (3, 3),
            'state_to_state_kernel_size' : (3, 3)
            }, return_state=False, return_sequence=True)
        
        self.channel_attention_0 = CALayer(64)
        
        self.convGRU2 = MultiConvGRU(cell_param={
            'input_channels' : 64,
            'output_channels' : 128,
            'input_to_state_kernel_size' : (3, 3),
            'state_to_state_kernel_size' : (3, 3)
            }, return_state=False, return_sequence=True)
        
        self.channel_attention_1 = CALayer(128)
        
        self.convGRU3 = MultiConvGRU(cell_param={
            'input_channels' : 128,
            'output_channels' : 256,
            'input_to_state_kernel_size' : (3, 3),
            'state_to_state_kernel_size' : (3, 3)
            }, return_state=False, return_sequence=False)
        
        self.channel_attention_2 = CALayer(256)
        
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.fc1 = nn.Linear(256 * 3 * 3, 128)
        self.fc2 = nn.Linear(53 + 128, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, wide, deep):
        
        temp_deep_list = []
        for i in range(deep.shape[1]):
            temp_deep = deep[:, i, :, :, :]
            temp_deep = temp_deep + self.channel_attention(temp_deep)
            temp_deep_list.append(temp_deep)

        deep = torch.stack(temp_deep_list, dim=1)
        
        deep = self.convGRU1(deep)
        

        temp_deep_list = []
        for i in range(deep.shape[1]):
            temp_deep = deep[:, i, :, :, :]
            temp_deep = temp_deep + self.channel_attention_0(temp_deep)
            temp_deep = self.pool(temp_deep)
            temp_deep_list.append(temp_deep)

        deep = torch.stack(temp_deep_list, dim=1)
        
        deep = self.convGRU2(deep)
        
        temp_deep_list = []
        for i in range(deep.shape[1]):
            temp_deep = deep[:, i, :, :, :]
            temp_deep = temp_deep + self.channel_attention_1(temp_deep)
            temp_deep = self.pool(temp_deep)
            temp_deep_list.append(temp_deep)

        deep = torch.stack(temp_deep_list, dim=1)
        
        _h = self.convGRU3(deep)
        
        _h = _h + self.channel_attention_2(_h)
        
        deep = self.pool(_h)
        
        deep = deep.view(-1, 256 * 3 * 3)
        
        deep = self.fc1(deep)
        wide = wide.view(-1, 53)
        wide_n_deep = torch.cat((wide, deep),1)
        
        wide_n_deep = F.relu(self.fc2(wide_n_deep))
        wide_n_deep = F.relu(self.fc3(wide_n_deep))
        return wide_n_deep

net = Net()

In [21]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# net = net.to(device)

# summary(net, [(1, 1, 1, 53),(4, 4, 31, 31)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
 AdaptiveAvgPool2d-1              [-1, 4, 1, 1]               0
            Conv2d-2              [-1, 2, 1, 1]              10
              ReLU-3              [-1, 2, 1, 1]               0
            Conv2d-4              [-1, 4, 1, 1]              12
           Sigmoid-5              [-1, 4, 1, 1]               0
 AdaptiveAvgPool2d-6              [-1, 4, 1, 1]               0
            Conv2d-7              [-1, 2, 1, 1]              10
              ReLU-8              [-1, 2, 1, 1]               0
            Conv2d-9              [-1, 4, 1, 1]              12
          Sigmoid-10              [-1, 4, 1, 1]               0
AdaptiveAvgPool2d-11              [-1, 4, 1, 1]               0
           Conv2d-12              [-1, 2, 1, 1]              10
             ReLU-13              [-1, 2, 1, 1]               0
           Conv2d-14              [-1, 

# Training

In [22]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [23]:
full_train_index = [*range(0, len(X_wide_train))]

for epoch in range(epochs):  # loop over the dataset multiple times
    starttime = datetime.datetime.now()
    train_index, val_index, _, _, = train_test_split(full_train_index,full_train_index,test_size=0.1)
    train_dataset = torch.utils.data.DataLoader(
        TrainLoader(X_wide_train[train_index], X_deep_train[train_index], y_train[train_index]), 
                                                 batch_size=batch_size,)
    val_dataset = torch.utils.data.DataLoader(
        TrainLoader(X_wide_train[val_index], X_deep_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size,)
    # training
    total_train_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_dataset):
        if torch.cuda.is_available():
            net.cuda()
            X_wide_train_cuda = batch_x[0].float().cuda()
            X_deep_train_cuda = batch_x[1].float().cuda()
            y_train_cuda = batch_y.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred_y = net(X_wide_train_cuda, X_deep_train_cuda)
        loss = criterion(pred_y, y_train_cuda)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    # validation
    total_val_loss = 0
    for _,(batch_val_x, batch_val_y) in enumerate(val_dataset):
        
        if torch.cuda.is_available():
            X_wide_val_cuda = batch_val_x[0].float().cuda()
            X_deep_val_cuda = batch_val_x[1].float().cuda()
            y_val_cuda = batch_val_y.cuda()
        
        pred_y = net(X_wide_val_cuda, X_deep_val_cuda)
        val_loss = criterion(pred_y, y_val_cuda)
        total_val_loss += val_loss.item()
    
        # print statistics
    if min_val_loss > total_val_loss:
        torch.save(net.state_dict(), model_name)
        min_val_loss = total_val_loss
    endtime = datetime.datetime.now()
    print('epochs [%d/%d] cost:%.2fs train_loss: %.5f val_loss: %.5f' % 
          (epoch + 1, epochs, (endtime-starttime).seconds, total_train_loss, total_val_loss))

print('Finished Training')

epochs [1/128] cost:25.00s train_loss: 7.85815 val_loss: 0.35427
epochs [2/128] cost:26.00s train_loss: 2.14582 val_loss: 0.14928
epochs [3/128] cost:25.00s train_loss: 1.19259 val_loss: 0.13098
epochs [4/128] cost:26.00s train_loss: 1.00862 val_loss: 0.12909
epochs [5/128] cost:25.00s train_loss: 0.99270 val_loss: 0.10852
epochs [6/128] cost:26.00s train_loss: 0.91470 val_loss: 0.11569
epochs [7/128] cost:26.00s train_loss: 0.91949 val_loss: 0.12824
epochs [8/128] cost:26.00s train_loss: 0.89364 val_loss: 0.09920
epochs [9/128] cost:26.00s train_loss: 0.80542 val_loss: 0.11395
epochs [10/128] cost:26.00s train_loss: 0.83544 val_loss: 0.09532
epochs [11/128] cost:26.00s train_loss: 0.79917 val_loss: 0.08974
epochs [12/128] cost:26.00s train_loss: 0.78400 val_loss: 0.09207
epochs [13/128] cost:26.00s train_loss: 0.74732 val_loss: 0.08111
epochs [14/128] cost:26.00s train_loss: 0.76885 val_loss: 0.12907
epochs [15/128] cost:26.00s train_loss: 0.81294 val_loss: 0.08455
epochs [16/128] cos

epochs [125/128] cost:26.00s train_loss: 16.95633 val_loss: 1.97991
epochs [126/128] cost:26.00s train_loss: 16.99985 val_loss: 1.97890
epochs [127/128] cost:25.00s train_loss: 16.99366 val_loss: 2.01402
epochs [128/128] cost:26.00s train_loss: 16.97661 val_loss: 1.98880
Finished Training


# Testing

In [24]:
# net.load_state_dict(torch.load(model_name))
years = test[5].unique()
test_list = []

for year in years:
    temp = test[test[5]==year]
    temp = temp.reset_index(drop=True)
    test_list.append(temp)
    
len(test_list)

4

In [25]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net()
net = net.to(device)
net.load_state_dict(torch.load(model_name))

<All keys matched successfully>

In [26]:

with torch.no_grad():
    for year, _test in zip(years, test_list):

        print(year, '年:')

        y_test_lat = _test.loc[:,3]
        
        y_test_long = _test.loc[:,4]
        
        X_wide_test = X_wide_scaler.transform(_test.loc[:,6:])

        final_test_list = []
        for ahead_time in ahead_times:
            year_test_list = []
            for pressure in pressures:
                scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
                X_deep = reanalysis_test_dict[scaler_name][reanalysis_test_dict[scaler_name][0].isin(_test[0].unique())].loc[:,5:]
                X_deep = X_deep_scaler_dict[scaler_name].transform(X_deep)
                X_deep_final = X_deep.reshape(-1, 1, 1, 31, 31)
                year_test_list.append(X_deep_final)
            X_deep_temp = np.concatenate(year_test_list, axis=2)
            final_test_list.append(X_deep_temp)
        X_deep_test = np.concatenate(final_test_list, axis=1)

        if torch.cuda.is_available():
            X_wide_test = Variable(torch.from_numpy(X_wide_test).float().cuda())
            X_deep_test = Variable(torch.from_numpy(X_deep_test).float().cuda())

        pred = net(X_wide_test, X_deep_test)

        pred = y_scaler.inverse_transform(pred.cpu().detach().numpy())

        pred_lat = pred[:,0]
        pred_long = pred[:,1]
        true_lat = y_test_lat
        true_long = y_test_long

        diff_lat = np.abs(pred_lat - true_lat)
        diff_long = np.abs(pred_long - true_long)

        print('avg lat:', sum(diff_lat)/len(diff_lat))
        print('avg long:', sum(diff_long)/len(diff_long))

        sum_error = []
        for i in range(0, len(pred_lat)):
            sum_error.append(great_circle((pred_lat[i], pred_long[i]), (true_lat[i], true_long[i])).kilometers)

        print('avg distance error:', sum(sum_error)/len(sum_error))

2015 年:
avg lat: 0.668006118436217
avg long: 0.8646890850319199
avg distance error: 125.21606908851712
2016 年:
avg lat: 0.8636830070511208
avg long: 0.9129902970327669
avg distance error: 145.4701460031522
2017 年:
avg lat: 0.7577311471981166
avg long: 0.9723578004276063
avg distance error: 141.04057444296916
2018 年:
avg lat: 0.8711327388327995
avg long: 1.0219548776782772
avg distance error: 152.9109631050797
